<a href="https://colab.research.google.com/github/hananbahtiti/Hybrid-Intrusion-detection-Systems/blob/main/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Download Dataset

In [ ]:
!wget -O NGIDS-DS.rar https://unsworks.unsw.edu.au/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/download

--2025-04-26 08:47:38--  https://unsworks.unsw.edu.au/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/download
Resolving unsworks.unsw.edu.au (unsworks.unsw.edu.au)... 54.253.215.118
Connecting to unsworks.unsw.edu.au (unsworks.unsw.edu.au)|54.253.215.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://unsworks.unsw.edu.au/server/api/core/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/content [following]
--2025-04-26 08:47:40--  https://unsworks.unsw.edu.au/server/api/core/bitstreams/0ac2820a-5131-43ab-90b2-c624c8d73649/content
Reusing existing connection to unsworks.unsw.edu.au:443.
HTTP request sent, awaiting response... 200 200
Length: unspecified [application/x-rar-compressed]
Saving to: ‘NGIDS-DS.rar’

NGIDS-DS.rar            [                <=> ] 941.51M  17.3MB/s    in 56s     

2025-04-26 08:48:36 (16.8 MB/s) - ‘NGIDS-DS.rar’ saved [987249484]



In [ ]:
!unrar x /content/NGIDS-DS.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/NGIDS-DS.rar

Creating    NGIDS                                                     OK
Creating    NGIDS/NGIDS-DS-v1                                         OK
Extracting  NGIDS/NGIDS-DS-v1/feature_descr.csv                            0%  OK 
Extracting  NGIDS/NGIDS-DS-v1/ground_truth.csv                             0%  OK 
Creating    NGIDS/NGIDS-DS-v1/host logs                               OK
Extracting  NGIDS/NGIDS-DS-v1/host logs/1.csv                              0%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/10.csv                             0%  1%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/11.csv                             1%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/12.csv                             1%  2%  OK 
Extracting  NGIDS/NGIDS-DS-v1/host logs/13.csv                             2%  OK 
Extrac

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Host data processing

In [ ]:
!pip install tqdm

In [ ]:
!rm -rf /content/NGIDS

In [3]:
import pandas as pd
import numpy as np
import os
import glob
import math
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

import time
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer


In [ ]:
!cp /content/final_del.csv /content/drive/MyDrive/hybrid_IDS/dataset/all/

In [6]:
class HostPreprocessing():
  def __init__(self):
    pass

  def __file_path_collection(self, folder_path, ends='.csv'):
    try:
      joined_list = []
      for file_name in os.listdir(folder_path):
        file_name = os.path.join(folder_path, file_name)
        if os.path.isfile(file_name) and file_name.endswith(ends):
          joined_list.append(file_name)
      return joined_list

    except Exception as e:
      return f"An error occurred: {e}"


  def rename_columns(self, folder_path):
    files = self.__file_path_collection(folder_path)
    try:
      for file in tqdm(files, desc="Renaming columns") :
        df = pd.read_csv(file)
        column_name = df.columns.tolist()
        df.rename(columns={
            column_name[0]:'Date',
            column_name[1]:'Time',
            column_name[2]:'Unique_Identification',
            column_name[3]:'Execution_Path',
            column_name[4]:'System_Calls_Identifiers',
            column_name[5]:"Event's_Unique_Identification",
            column_name[6]:'attacks',
            column_name[7]:"Sub_Type_Attack",
            column_name[8]:"Label"
            }, inplace=True)
        df.to_csv(file, index=False)
      return f"done files..."

    except Exception as e:
      return f"An error occurred: {e}"


  def data_encoding(self, folder_path, column_number : int):
    files = self.__file_path_collection(folder_path)
    encoder = LabelEncoder()
    all_value = []
    try:
      for file in tqdm(files, desc="Extract all values from files"):
        df = pd.read_csv(file)
        if column_number < df.shape[1]:
                all_value.extend(df.iloc[:, column_number].dropna().unique())

        else:
            print(f"❌ Skipped (no column {column_number}): {file}")

      print( f"number value: {len(all_value)}" )
      encoder.fit(all_value)

      for file in tqdm(files, desc="encoder data"):
        df = pd.read_csv(file)
        df.iloc[:, column_number] = encoder.transform(df.iloc[:, column_number])
        #df.drop(columns=[df.columns[column_number]], inplace=True)
        #print(len(df.iloc[:, column_number].dropna().unique()), df.iloc[:, column_number].dropna().unique(), file )
        df.to_csv(file, index=False)
      return f"done file..."

    except Exception as e:
      return f"An error occurred: {e}"








  def encode_execution_paths(folder_path: str, path_column: str = 'Execution_Path', encoding: str = 'utf-8'):
      """
      Encode execution paths in CSV files inside a folder by splitting the paths into parts.

      Args:
          folder_path (str): Path to the folder containing CSV files.
          path_column (str): Name of the column containing the paths. Default is 'Execution_Path'.
          encoding (str): Encoding of the CSV files. Default is 'utf-8'.
      """

      # Get all CSV filenames
      csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

      # Step 1: Build the dictionary and find maximum path depth
      unique_parts = set()
      max_depth = 0

      print("Step 1/2: Building the dictionary and detecting path depth...")
      for filename in tqdm(csv_files, desc="Building dictionary"):
          file_path = os.path.join(folder_path, filename)

          for chunk in pd.read_csv(file_path, usecols=[path_column], chunksize=10000, encoding=encoding):
              paths = chunk[path_column].dropna()
              for path in paths:
                  parts = str(path).split('/')
                  parts = [p for p in parts if p]
                  unique_parts.update(parts)
                  max_depth = max(max_depth, len(parts))

      # Create the encoder dictionary
      encoder = {part: idx for idx, part in enumerate(sorted(unique_parts))}

      print(f"Number of unique parts: {len(encoder)}")
      print(f"Maximum path depth: {max_depth}")

      # Step 2: Process each file and merge results
      print("\nStep 2/2: Processing files and merging results...")
      for filename in tqdm(csv_files, desc="Processing files"):
          file_path = os.path.join(folder_path, filename)
          new_chunks = []

          for chunk in pd.read_csv(file_path, chunksize=10000, encoding=encoding):
              if path_column not in chunk.columns:
                  continue

              encoded_columns = []

              for path in chunk[path_column].fillna(''):
                  parts = str(path).split('/')
                  parts = [p for p in parts if p]
                  encoded_parts = [encoder.get(part, np.nan) for part in parts]
                  while len(encoded_parts) < max_depth:
                      encoded_parts.append(np.nan)
                  encoded_columns.append(encoded_parts)

              # Convert encoded parts to DataFrame with improved column names
              encoded_df = pd.DataFrame(
                  encoded_columns,
                  columns=[f"{path_column}_part_{i}" for i in range(max_depth)]
              )

              # Merge original data with encoded data
              combined_chunk = pd.concat([chunk.reset_index(drop=True), encoded_df], axis=1)
              new_chunks.append(combined_chunk)

          # Merge all chunks
          final_df = pd.concat(new_chunks, ignore_index=True)

          # Save the final file
          new_file_path = os.path.join(folder_path, filename)
          final_df.to_csv(new_file_path, index=False, encoding=encoding)

      print("\n✅ Done! All files have been processed and saved successfully.")






  def extract_datetime_features(self, folder_path, date_col, time_col ):
    files = self.__file_path_collection(folder_path) # Assign the output of __file_path_collection to files
    try:
      for file in tqdm(files, desc="read all files"): # Iterate through the 'files' list, not 'file_paths'
        df = pd.read_csv(file) # Read each file into a DataFrame
        if date_col in df.columns and time_col in df.columns :
          df[date_col] = pd.to_datetime(df[date_col], format='%d/%m/%Y', errors='coerce', dayfirst=True)

          df['year'] = df[date_col].dt.year
          df['month'] = df[date_col].dt.month
          df['day'] = df[date_col].dt.day

          # Assuming 'Time' column is string and needs to be converted to datetime
          df[time_col] = pd.to_datetime(df[time_col], format='%H:%M:%S', errors='coerce', dayfirst=True)

          df['hour'] = df[time_col].dt.hour
          df['minute'] = df[time_col].dt.minute
          df['second'] = df[time_col].dt.second

          df.drop(columns=[date_col, time_col], inplace=True)

          df.to_csv(file, index=False) # Save changes back to the original file
          print(f"✅ Processed: {df.head(3)}")

    except Exception as e:
        print(f"❌ Error in {file}: {e}") # Print the error and the file name








  def file_collection(self, folder_path):
    try:
      file_paths = self.__file_path_collection(folder_path)
      if file_paths:
        output_folder = os.path.join(os.getcwd(),'files')
        os.makedirs(output_folder ,exist_ok=True)
        output_file = os.path.join(output_folder,"final.csv")

        for file in tqdm(file_paths, desc="read all files"):

          first = True
          chunk = pd.read_csv(file, chunksize=10000)
          for part in chunk:
            part.to_csv(output_file, mode='a', header=first, index=False)
            first = False

      return  f"done file..."

    except Exception as e:
      return f"An error occurred: {e}"



  def delete_columns(self, file_path, columns_names: list):
    try:
        # اقرأ الملف كدفعات
        chunks = pd.read_csv(file_path, chunksize=10000)

        # ملف مؤقت للحفظ
        temp_file_path = file_path.replace('.csv', '_temp.csv')

        first_chunk = True

        for chunk in chunks:
            # حذف الأعمدة المطلوبة إن وجدت
            chunk = chunk.drop(columns=[col for col in columns_names if col in chunk.columns], axis=1)

            # حفظ إلى الملف المؤقت
            chunk.to_csv(temp_file_path, mode='w' if first_chunk else 'a', header=first_chunk, index=False)
            first_chunk = False

        os.remove(file_path)
        os.rename(temp_file_path, file_path)  # إعادة تسمية الملف المؤقت

        return "✅ Columns deleted and file overwritten successfully!"

    except Exception as e:
        return f"❌ An error occurred: {e}"




  def smart_impute_and_save(self, input_file, output_file, chunk_size=1000000):
    """
    This method automatically fills missing values based on their missing ratio,
    then saves the cleaned data to a new CSV file.
    """
    try:
        start_time = time.time()  # Start timing ⏱️

        # Load the data
        chunks = pd.read_csv(input_file, chunksize=chunk_size)
        df = next(chunks)  # Read the first chunk

        # Define the target columns for imputation
        cols_to_impute = ['Execution_Path_part_2', 'Execution_Path_part_3', 'Execution_Path_part_4']

        # Build the feature set including helper columns
        features = df[['attacks', 'Sub_Type_Attack', 'Label', 'Execution_Path_part_1'] + cols_to_impute]

        # Calculate missing value ratios
        missing_ratio = features.isna().mean()
        print("📊 Missing value ratio per column:\n", missing_ratio)

        # Smart imputation based on missing value ratio
        for col in tqdm(cols_to_impute, desc="Imputing columns", unit="column"):  # Adding tqdm here for progress bar
            if col not in df.columns:
                continue  # Skip if the column was already dropped

            ratio = missing_ratio[col]
            print(f"\n=== Processing column: {col} (Missing {ratio:.2%}) ===")

            if ratio > 0.8:
                print(f"⚠️  Column {col} has too many missing values (>80%). Dropping it...")
                df.drop(columns=[col], inplace=True)

            elif ratio > 0.3:
                print(f"🛠️  Using Iterative Imputer for {col}...")
                imputer = IterativeImputer(max_iter=10, random_state=0)
                temp_data = features[['attacks', 'Sub_Type_Attack', 'Label', 'Execution_Path_part_1', col]]

                # Wrap the imputation in tqdm to show the progress per iteration
                for i in tqdm(range(imputer.max_iter), desc=f"Iterating {col}", leave=False, unit="iteration"):
                    imputed_data = imputer.fit_transform(temp_data)
                df[col] = imputed_data[:, -1]

            else:
                print(f"🔍 Using KNN Imputer for {col}...")
                imputer = KNNImputer(n_neighbors=5)
                temp_data = features[['attacks', 'Sub_Type_Attack', 'Label', 'Execution_Path_part_1', col]]

                # KNN imputation is done in one step, so we don't need a loop for iterations
                imputed_data = imputer.fit_transform(temp_data)
                df[col] = imputed_data[:, -1]

        # Save the cleaned data
        df.to_csv(output_file, index=False)
        print(f"\n✅ Successfully completed missing value filling and saved to {output_file}.")

        end_time = time.time()  # End timing ⏱️
        elapsed_time = end_time - start_time
        print(f"⏱️ Total execution time: {elapsed_time:.2f} seconds")

        return df

    except Exception as e:
        return f"❌ An error occurred: {e}"


a = HostPreprocessing()
#a.file_path_collection("/content/NGIDS/NGIDS-DS-v1/host logs")
#a.file_collection("/content/host logs")
#a.rename_columns( '/content/NGIDS/NGIDS-DS-v1/host logs')
#a.data_encoding('/content/NGIDS/NGIDS-DS-v1/host logs', column_number=6)
#a.data_encoding('/content/NGIDS/NGIDS-DS-v1/host logs', column_number=7)

#c=a.encode_execution_paths('/content/host logs', path_column='Execution_Path')


#a.extract_datetime_features(  folder_path='/content/NGIDS/NGIDS-DS-v1/host logs'
#                           ,date_col='Date',
 #                           time_col='Time' )
#a.delete_data( folder_path='/content/NGIDS/NGIDS-DS-v1/host logs')
#a.delete_columns( file_path='/content/final.csv' ,
#                 columns_names=['Unique_Identification', 'System_Calls_Identifiers',
 #                               "Event's_Unique_Identification", 'hour', 'minute','second',
 #                               'Execution_Path_part_0'])
a.smart_impute_and_save(input_file='/content/drive/MyDrive/hybrid_IDS/dataset/all/final_del.csv',
                         output_file='/content/final_nan.csv')

📊 Missing value ratio per column:
 attacks                  0.000000
Sub_Type_Attack          0.000000
Label                    0.000000
Execution_Path_part_1    0.000000
Execution_Path_part_2    0.012290
Execution_Path_part_3    0.946074
Execution_Path_part_4    0.985485
dtype: float64


Imputing columns:   0%|          | 0/3 [00:00<?, ?column/s]


=== Processing column: Execution_Path_part_2 (Missing 1.23%) ===
🔍 Using KNN Imputer for Execution_Path_part_2...


Imputing columns: 100%|██████████| 3/3 [11:32<00:00, 230.95s/column]



=== Processing column: Execution_Path_part_3 (Missing 94.61%) ===
⚠️  Column Execution_Path_part_3 has too many missing values (>80%). Dropping it...

=== Processing column: Execution_Path_part_4 (Missing 98.55%) ===
⚠️  Column Execution_Path_part_4 has too many missing values (>80%). Dropping it...

✅ Successfully completed missing value filling and saved to /content/final_nan.csv.
⏱️ Total execution time: 694.05 seconds


,attacks,Sub_Type_Attack,Label,Execution_Path_part_1,Execution_Path_part_2
0,7,52,0,16,57.0
1,7,52,0,79,113.0
2,7,52,0,16,21.0
3,7,52,0,79,55.0
4,7,52,0,16,132.0
...,...,...,...,...,...
999995,7,52,0,102,5.0
999996,7,52,0,102,5.0
999997,7,52,0,16,21.0
999998,7,52,0,102,5.0


In [ ]:
import pandas as pd
file = '/content/final.csv'

chunk_size = 1000000  # عدد الصفوف بالدفعة
chunks = pd.read_csv(file, chunksize=chunk_size)

for chunk in chunks:
  print(chunk.info())
  print(chunk.head())
  break

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 7 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   attacks                1000000 non-null  int64  
 1   Sub_Type_Attack        1000000 non-null  int64  
 2   Label                  1000000 non-null  int64  
 3   Execution_Path_part_1  1000000 non-null  int64  
 4   Execution_Path_part_2  987710 non-null   float64
 5   Execution_Path_part_3  53926 non-null    float64
 6   Execution_Path_part_4  14515 non-null    float64
dtypes: float64(3), int64(4)
memory usage: 53.4 MB
None
   attacks  Sub_Type_Attack  Label  Execution_Path_part_1  \
0        7               52      0                     16   
1        7               52      0                     79   
2        7               52      0                     16   
3        7               52      0                     79   
4        7               52      0        

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

file = '/content/final.csv'

# Load the data
chunk_size = 1000000  # عدد الصفوف بالدفعة
chunks = pd.read_csv(file, chunksize=chunk_size)
target_col = 'Label'

for chunk in chunks:
    # حساب مصفوفة الارتباط فقط بالنسبة للعمود المستهدف (Label)
    corr = chunk.corr(method='pearson')[target_col]
    #corr = chunk.corr(method='pearson')

    # عرض المصفوفة كجدول مرتب
    from IPython.display import display  # لو تعمل داخل jupyter أو colab
    display(corr)

    # عرض المصفوفة الحرارية (heatmap) مع اللون
    plt.figure(figsize=(10, 8), dpi=500)
    sns.heatmap(corr.T, annot=True, fmt=".2f", linewidth=.5, cmap='coolwarm', vmin=-1, vmax=1)
    plt.title(f"Correlation Matrix with {target_col}")
    plt.show()

    break  # فقط أول دفعة


,Label
Unique_Identification,0.008000
System_Calls_Identifiers,0.000824
Event's_Unique_Identification,-0.044237
attacks,-0.988050
Sub_Type_Attack,-0.868515
Label,1.000000
hour,-0.055726
minute,0.020046
second,-0.030015
Execution_Path_part_0,-0.011172


IndexError: Inconsistent shape between the condition and the input (got (14, 1) and (14,))

<Figure size 5000x4000 with 0 Axes>

In [ ]:
!cp -r "/content/host logs" "/content/drive/MyDrive/hybrid_IDS/dataset/all/"

In [ ]:
!cp -r /content/files/final.csv /content/drive/MyDrive/hybrid_IDS/dataset/all/

In [ ]:
!rm -rf "/content/host logs"

In [ ]:
correlation_with_target = numeric_df.corr()[target_col].drop(target_col).sort_values(ascending=False)
correlation_with_target

,Label
path_part_4,0.259518
path_part_5,0.128456
path_part_2,0.122674
hour,0.010651
path_part_1,0.008840
Event's_Unique_Identification,0.005089
Unique_Identification,0.002967
second,0.001407
System_Calls_Identifiers,-0.001347
minute,-0.011156


In [ ]:
import pandas as pd


file = '/content/final.csv'
# Load the data
chunk_size = 1000000  # عدد الصفوف بالدفعة
chunks = pd.read_csv(file, chunksize=chunk_size)
target_col = 'Label'
for chunk in chunks:
    #print(chunk.info())
    #print (chunk.head())
    correlation_with_target = chunk.corr()[target_col].drop(target_col).sort_values(ascending=False)
    #print(correlation_with_target)
    # Find the pearson correlations matrix
    #corr = chunk.corr(method = 'pearson')
    #print(corr)
    print(chunk['year'].value_counts())
    print(chunk['month'].value_counts())
    print(chunk['day'].value_counts())

    break  # فقط لعرض أول دفعة كمثال

year
2016    1000000
Name: count, dtype: int64
month
3    1000000
Name: count, dtype: int64
day
15    1000000
Name: count, dtype: int64


In [ ]:
import pandas as pd

file = '/content/final.csv'

# Load the data
chunk_size = 1000000  # عدد الصفوف بالدفعة
chunks = pd.read_csv(file, chunksize=chunk_size)
target_col = 'Label'

for chunk in chunks:
    # حساب مصفوفة الارتباط (Pearson)
    corr = chunk.corr(method='pearson')



    # عرض المصفوفة كجدول مرتب
    from IPython.display import display  # لو تعمل داخل jupyter أو colab
    display(corr)

    break  # فقط أول دفعة


,Unique_Identification,System_Calls_Identifiers,Event's_Unique_Identification,attacks,Sub_Type_Attack,Label,hour,minute,second,Execution_Path_part_0,Execution_Path_part_1,Execution_Path_part_2,Execution_Path_part_3,Execution_Path_part_4
Unique_Identification,1.000000,-0.356043,-0.226277,-0.006008,-0.007618,0.008000,-0.174586,-0.074778,0.031213,0.018835,0.835435,-0.180804,0.222207,0.424684
System_Calls_Identifiers,-0.356043,1.000000,-0.020682,-0.000216,0.000920,0.000824,-0.026462,0.002737,0.031236,-0.008794,-0.322491,0.162257,0.125857,0.156079
Event's_Unique_Identification,-0.226277,-0.020682,1.000000,0.047582,0.051690,-0.044237,0.871884,0.141818,0.032540,0.012869,-0.253982,-0.045786,-0.024917,-0.182539
attacks,-0.006008,-0.000216,0.047582,1.000000,0.852407,-0.988050,0.060558,-0.024268,0.030432,0.014902,-0.054266,-0.151590,0.319476,-0.676846
Sub_Type_Attack,-0.007618,0.000920,0.051690,0.852407,1.000000,-0.868515,0.059781,-0.015544,0.040103,0.019171,-0.032661,-0.071744,0.362865,-0.411986
Label,0.008000,0.000824,-0.044237,-0.988050,-0.868515,1.000000,-0.055726,0.020046,-0.030015,-0.011172,0.057090,0.152719,-0.324545,0.679153
hour,-0.174586,-0.026462,0.871884,0.060558,0.059781,-0.055726,1.000000,-0.331144,0.018938,0.005996,-0.210649,-0.030925,-0.042714,-0.235855
minute,-0.074778,0.002737,0.141818,-0.024268,-0.015544,0.020046,-0.331144,1.000000,0.006425,0.005650,-0.070744,-0.016027,0.037412,0.060329
second,0.031213,0.031236,0.032540,0.030432,0.040103,-0.030015,0.018938,0.006425,1.000000,0.084023,0.022400,-0.072040,-0.067048,0.333718
Execution_Path_part_0,0.018835,-0.008794,0.012869,0.014902,0.019171,-0.011172,0.005996,0.005650,0.084023,1.000000,0.020558,NaN,NaN,NaN


#network




In [ ]:
pip install scapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 14.3 MB/s eta 0:00:00


In [ ]:
import csv
import json
import os
import re
from tqdm import tqdm
from scapy.all import PcapReader

In [ ]:
class NetworkPreprocessing:
  def __init__(self):
    pass


  def extract_tcp_option(self, option, key):
    for opt in option:
      if opt[0] == key:
        return opt[1]
    return None


  def delete_data(self, file_path):
    df = pd.read_csv(file_path)
    df = df.loc[:, df.nunique() > 1]
    df.to_csv(file_path, index=False)
    return f"Columns containing only one value are deleted..."

  def convert_pcap_csv(self, pcap_file, csv_file):
    try:
      print("Countig packages...")
      with PcapReader(pcap_file) as count_reader:
        total_packets = sum(1 for _ in count_reader)
        print(f"Total number of packages: {total_packets}")

      print("Start converting to csv file...")
      with PcapReader(pcap_file) as packets:
        with open(csv_file, mode='w', newline="") as file:
          writer = csv.writer(file)
          writer.writerow([
              "Packet Number", "Ethernet DST", "Ethernet SRC", "Ethernet Type",
              "IP Version", "IP IHL", "IP TOS", "IP Length", "IP ID", "IP Flags",
              "IP Fragmentation", "IP TTL", "IP Proto", "IP Checksum",
              "IP Src", "IP Dst", "TCP Src Port", "TCP Dst Port", "TCP Seq",
              "TCP Ack", "TCP Data Offset", "TCP Reserved", "TCP Flags",
              "TCP Window", "TCP Checksum", "TCP Urgent Pointer", "TCP Options MSS",
              "TCP Options WScale", "TCP Options NOP","Hexdump"
          ])

          for i, packet in enumerate(tqdm(packets,total=total_packets, desc="Packet processing")):
            if packet.haslayer('Ethernet'):
              eth_dst = packet['Ethernet'].dst
              eth_src = packet['Ethernet'].src
              eth_type = packet['Ethernet'].type
            else:
              eth_dst = eth_src = eth_type = None

            if packet.haslayer('IP'):
              ip_version = packet['IP'].version
              ip_ihl = packet['IP'].ihl
              ip_tos = packet['IP'].tos
              ip_len = packet['IP'].len
              ip_id = packet['IP'].id
              ip_flags = packet['IP'].flags
              ip_frag = packet['IP'].frag
              ip_ttl = packet['IP'].ttl
              ip_proto = packet['IP'].proto
              ip_chksum = packet['IP'].chksum
              ip_src = packet['IP'].src
              ip_dst = packet['IP'].dst
            else:
              ip_version = ip_ihl = ip_tos = ip_len = ip_id = ip_flags = ip_frag = ip_ttl = ip_proto = ip_chksum = ip_src = ip_dst = None

            if packet.haslayer('TCP'):
              tcp_sport = packet['TCP'].sport
              tcp_dport = packet['TCP'].dport
              tcp_seq = packet['TCP'].seq
              tcp_ack = packet['TCP'].ack
              tcp_dataofs = packet['TCP'].dataofs
              tcp_reserved = packet['TCP'].reserved
              tcp_flags = packet['TCP'].flags
              tcp_window = packet['TCP'].window
              tcp_chksum = packet['TCP'].chksum
              tcp_urgptr = packet['TCP'].urgptr
              tcp_options = packet['TCP'].options

              tcp_mss = self.extract_tcp_option(tcp_options, 'MSS')
              tcp_wscale = self.extract_tcp_option(tcp_options, 'WScale')
              tcp_nop_count = self.extract_tcp_option(tcp_options, 'NOP')

            else:
              tcp_sport = tcp_dport = tcp_seq = tcp_ack = tcp_dataofs = tcp_reserved = tcp_flags = tcp_window = tcp_chksum = tcp_urgptr = tcp_mss = tcp_wscale = tcp_nop_count = None

            raw_data = packet.original.hex()
            writer.writerow([
                i + 1, eth_dst, eth_src, eth_type,
                  ip_version, ip_ihl, ip_tos, ip_len, ip_id, ip_flags,
                  ip_frag, ip_ttl, ip_proto, ip_chksum,
                  ip_src, ip_dst, tcp_sport, tcp_dport, tcp_seq,
                  tcp_ack, tcp_dataofs, tcp_reserved, tcp_flags,
                  tcp_window, tcp_chksum, tcp_urgptr, tcp_mss, tcp_wscale, tcp_nop_count,
                  raw_data
            ])

      return f"The file has been converted to csv successfully."
    except Exception as e:
      return f"An error occurred: {e}"

a = NetworkPreprocessing()
#a.convert_pcap_csv(pcap_file='/content/NGIDS/NGIDS-DS-v1/NGIDS.pcap',
#                   csv_file='/content/drive/MyDrive/hybrid_IDS/network.csv')
#a.delete_data(file_path='/content/drive/MyDrive/hybrid_IDS/network.csv')

'Columns containing only one value are deleted...'

In [ ]:
class PcapFileProcessing:
  def __init__(self) -> None:
    pass



  def delete_data(self, file_path):
    df = pd.read_csv(file_path)
    df = df.loc[:, df.nunique() > 1]
    df.to_csv(file_path, index=False)
    return f"Columns containing only one value are deleted..."


  def date_time_columns(self, file_path, csv_file):
    try:
        df = pd.read_csv(file_path, dtype=str)
        df.columns = df.columns.str.strip()
        if 'timestamp' not in df.columns:
            return "Error: The 'timestamp' column is missing from the file."

        df['timestamp'] = df['timestamp'].astype(str).str.strip()

        df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce', dayfirst=True)

        if df['timestamp'].isnull().all():
            return "All values in 'timestamp' are invalid and could not be converted."

        # Extract date and time
        df['date'] = df['timestamp'].dt.strftime('%d/%m/%Y')
        df['time'] = df['timestamp'].dt.strftime('%H:%M:%S')

        # Save the result
        df.to_csv(csv_file, index=False)

        return "Successfully converted 'timestamp' to 'date' and 'time'."

    except Exception as e:
        return f"An error occurred during conversion: {e}"





  def convert_log_csv(self, log_file, csv_file):
    try:
      print("Start converting to csv file...")
      with open(log_file, "r") as logfile:
        log_data = logfile.read()
      pattern = re.compile(
          r'(?P<timestamp>\d{2}/\d{2}/\d{4}-\d{2}:\d{2}:\d{2}\.\d+)\s+\[\*\*\]\s+\[(?P<sid>[^]]+)\]\s+(?P<signature>.*?)\s+\[\*\*\]\s+\[Classification:\s+(?P<classification>.*?)\]\s+\[Priority:\s+(?P<priority>\d+)\]\s+\{(?P<protocol>\w+)\}\s+(?P<src_ip>\d+\.\d+\.\d+\.\d+):(?P<src_port>\d+)\s+->\s+(?P<dst_ip>\d+\.\d+\.\d+\.\d+):(?P<dst_port>\d+)'
          )
      with open(csv_file, "w", newline="") as csvfile:
        fieldnames = ['timestamp', 'sid', 'signature', 'classification',
                      'priority', 'protocol','src_ip', 'src_port', 'dst_ip', 'dst_port']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for match in pattern.finditer(log_data):
          writer.writerow(match.groupdict())

      return f"The file has been converted to csv successfully."

    except Exception as e:
      return f"An error occurred: {e}"


  def convert_json_csv(self, json_file, csv_file):
    try:
      print("Start converting JSON to CSV...")
      with open(json_file, 'r') as f:
        data = [json.loads(line) for line in f if line.strip()]

      fieldnames = set()
      for row in data:
          fieldnames.update(row.keys())

      fieldnames = list(fieldnames)

      with open(csv_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(fieldnames)

        for row in data:
          writer.writerow(row.get(field, '') for field in fieldnames)

      return f"The file has been converted to csv successfully."

    except Exception as e:
      return f"An error occurred: {e}"



a = PcapFileProcessing()
#a.convert_log_csv( log_file="/content/drive/MyDrive/hybrid_IDS/fast.log",
#                  csv_file="/content/drive/MyDrive/hybrid_IDS/alerts.csv")

#a.convert_json_csv(json_file='/content/drive/MyDrive/hybrid_IDS/eve.json',
#                   csv_file='/content/drive/MyDrive/hybrid_IDS/eve.csv')

#a.delete_data( file_path='/content/drive/MyDrive/hybrid_IDS/eve.csv')
a.date_time_columns(file_path='/content/drive/MyDrive/hybrid_IDS/eve.csv',
                    csv_file='/content/drive/MyDrive/hybrid_IDS/dataset/eve.csv')

'تم تحويل timestamp بنجاح إلى date و time.'